# Imports

In [2]:
import tensorflow as tf
import time

import numpy as np # linear algebra
import matplotlib.pyplot as plt
import pandas as pd
from copy import copy

master_df = pd.read_csv('../data/us_kickstarter.zip')
df = copy(master_df)


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Preprocessing

In [4]:
features_to_select = ['category', 'category_parent_id', 'days_to_deadline', 'staff_pick', \
                 'location_type', 'location_state', 'location_id', 'goal', \
                  'launched_weekday', 'description_length', 'is_weekend_launch', \
                 'launched_week', 'launched_month']


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

def clean_words (word):
    word = word.lower()
    word_list = word.split()
    clean_words = [word for word in word_list if not word.isdigit()]
    return " ".join(clean_words)

df['cleaned_description'] = df['blurb'].fillna("").apply(clean_words)
# df['cleaned_description'] = df['cleaned_description'].apply(stem_description)

tfidf = TfidfVectorizer(max_features = 500, use_idf = True, \
                        stop_words='english', ngram_range=(1,2))

X = tfidf.fit_transform(df['cleaned_description'].values)

vectors = pd.DataFrame(X.toarray(), columns=['_' + w for w in tfidf.get_feature_names()])

In [6]:
df = pd.concat([df, vectors], axis=1)

In [7]:
features_to_select.extend(vectors.columns)

In [8]:
df = df[df['state'] != 'live']

In [9]:
df['state'] = np.where(df['state'] == 'successful', 1, 0)

In [10]:
target = df['state']
df = df[features_to_select]

In [11]:
le_category = LabelEncoder()
le_location_type = LabelEncoder()
le_location_state = LabelEncoder()

In [12]:
df['category'] = le_category.fit_transform(df['category'])

df['location_type'] = le_location_type.fit_transform(df['location_type'])

df['location_state'] = le_location_state.fit_transform(df['location_state'])

In [13]:
df['category_parent_id'] = df['category_parent_id'].fillna(0)

In [14]:
to_scale = ['goal', 'description_length', 'days_to_deadline']

In [15]:
scaler = StandardScaler()

In [16]:
Xtrain, Xtest, ytrain, ytest = train_test_split(df, target, test_size=0.3, random_state=42)

In [17]:
Xval, Xtest, yval, ytest = train_test_split(Xtest, ytest, test_size = 0.4, random_state=42)

In [18]:
to_scale = ['goal', 'description_length', 'days_to_deadline']
scaler = StandardScaler()
Xtrain.loc[:,to_scale] = scaler.fit_transform(Xtrain[to_scale])
Xval.loc[:,to_scale] = scaler.transform(Xval[to_scale])
Xtest.loc[:,to_scale] = scaler.transform(Xtest[to_scale])

C:\Users\sahil\Anaconda3\envs\tf_gpu\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [19]:
Xtrain = Xtrain.values
Xval = Xval.values
Xtest = Xtest.values

In [20]:
ytrain = ytrain.values
yval = yval.values
ytest = ytest.values

In [21]:
# one hot encode labels
def one_hot_encode(y_):
    z = np.zeros((y_.shape[0], 2))
    for i in range(y_.shape[0]):
        z[i][y_[i]] = 1
    return z

In [22]:
ytrain = one_hot_encode(ytrain)

yval = one_hot_encode(yval)
ytest = one_hot_encode(ytest)

# Parameters

In [23]:
# Global Parameters
#--------------------------------
# learning rate
learning_rate = 0.05

#training_epochs = 1000
#batch_size = 30

training_epochs = 10
batch_size = 50

display_step = 2

#Network Architecture
# -----------------------------------------
#
# Two hidden layers
#
#------------------------------------------
# number of neurons in layer 1
n_hidden_1 = 1000
# number of neurons in layer 2
n_hidden_2 = 600
# number of neurons in layer 3
n_hidden_3 = 300
# number of neurons in layer 4
n_hidden_4 = 100
# number of neurons in layer 5
n_hidden_5 = 25

#MNIST data image of shape 28*28=784
input_size = 513

# 0-9 digits recognition (labels)
output_size = 2

# Define Layer Function

In [24]:
def layer(x, weight_shape, bias_shape):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    # comes from the study by He et al. for ReLU layers
    w_std = (2.0/weight_shape[0])**0.5
    #print(weight_shape[0])
    #w_std = 0.5;

    #initialization of the weights
    #you can try either
    w_0 = tf.random_normal_initializer(stddev=w_std)
    #w_0 = tf.random_uniform_initializer(minval=-1,maxval=1)

    b_0 = tf.constant_initializer(value=0)
    
    W = tf.get_variable("W", weight_shape, initializer=w_0)
    b = tf.get_variable("b", bias_shape,   initializer=b_0)
    
    print('Weight Matrix:', W)
    print('Bias Vector:', b)
    
    # different activation functions
    # you can try
    # (1) linear activation (not a good idea)
    #return tf.matmul(x, W) + b
    # (2) tanh activation
    #return tf.nn.tanh(tf.matmul(x, W) + b)
    # (3) sigmoid activation
    #return tf.nn.sigmoid(tf.matmul(x, W) + b)
    # (4) relu activation
    return tf.nn.sigmoid(tf.matmul(x, W) + b)


# Define Inference Function

In [25]:
def inference(x):
    """
    define the whole network (5 hidden layers + output layers)
    input:
        - a batch of pictures 
        (input shape = (batch_size*image_size))
    output:
        - a batch vector corresponding to the logits predicted by the network
        (output shape = (batch_size*output_size)) 
    """
    
    with tf.variable_scope("hidden_layer_1"):
        hidden_1 = layer(x, [input_size, n_hidden_1], [n_hidden_1])
        #print([input_size, n_hidden_1])
     
    with tf.variable_scope("hidden_layer_2"):
        hidden_2 = layer(hidden_1, [n_hidden_1, n_hidden_2], [n_hidden_2])
        #print([n_hidden_1, n_hidden_2])
        
    with tf.variable_scope("hidden_layer_3"):
        hidden_3 = layer(hidden_2, [n_hidden_2, n_hidden_3], [n_hidden_3])
        #print([n_hidden_2, n_hidden_3])
        
    with tf.variable_scope("hidden_layer_4"):
        hidden_4 = layer(hidden_3, [n_hidden_3, n_hidden_4], [n_hidden_4])
        #print([n_hidden_3, n_hidden_4])
        
    with tf.variable_scope("hidden_layer_5"):
        hidden_5 = layer(hidden_4, [n_hidden_4, n_hidden_5], [n_hidden_5])
        #print([n_hidden_4, n_hidden_5])
     
    with tf.variable_scope("output"):
        output = layer(hidden_5, [n_hidden_5, output_size], [output_size])
        #print([n_hidden_5, output_size])

    return output

# Define Loss Function

## Define First Loss 

In [26]:
def loss_1(output, y):
    """
    computes the average error per data sample 
    by computing the cross-entropy loss over a minibatch
    intput:
        - output: the output of the inference function 
        - y: true value of the sample batch
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    dot_product = y * tf.log(output)
    
    #tf.reduce_sum: Computes the sum of elements across dimensions of a tensor.
    xentropy = -tf.reduce_sum(dot_product, 1)
    
    #tf.reduce_mean: Computes the mean of elements across dimensions of a tensor.
    loss = tf.reduce_mean(xentropy)

    return loss

## Define Second Loss

In [27]:
def loss_2(output, y):
    """
    Computes softmax cross entropy between logits and labels and then the loss 
    
    intput:
        - output: the output of the inference function 
        - y: true value of the sample batch
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """

    #mean square error
    #loss = tf.reduce_mean(tf.reduce_sum(tf.square(y-output)))
    
    #Computes softmax cross entropy between logits and labels.
    xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y)
    loss = tf.reduce_mean(xentropy)

    return loss

# Define the optimizer and training target

In [28]:
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """

    tf.summary.scalar("cost", cost)
    # tf.train.GradientDescentOptimizer
    # will use different optimization routimes 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)

    return train_op

# Define evaluation method

In [29]:
def evaluate(output, y):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -y: true value for the validation set
    output:
        - accuracy: accuracy on the validation set (scalar between 0 and 1)
    """
    #correct prediction is a binary vector which equals one when the output and y match
    #otherwise the vector equals 0
    #tf.cast: change the type of a tensor into another one
    #then, by taking the mean of the tensor, we directly have the average score, so the accuracy
    
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    tf.summary.scalar("validation_error", (1.0 - accuracy))

    return accuracy

# Main function

In [30]:
if __name__ == '__main__':
    
    start_time = time.time()
    

    with tf.Graph().as_default():
        
        with tf.variable_scope("multi_da_layer"):
            #neural network definition 
            
            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label
            x = tf.placeholder("float", [None, input_size])   # MNIST data image of shape 28*28=784
            y = tf.placeholder("float", [None, output_size])  # 0-9 digits recognition

            #the network is defined using the inference function defined above in the code
            output = inference(x)
            #calculate loss with tensorflow's cross-entropy function
            cost = loss_2(output, y)
            
            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)
            
            train_op = training(cost, global_step)
            #train_op = training(cost, global_step=None)
            
            #evaluate the accuracy of the network (done on a validation set)
            eval_op = evaluate(output, y)

            summary_op = tf.summary.merge_all()
    
            #save and restore variables to and from checkpoints.
            saver = tf.train.Saver()
    
            #defines a session
            sess = tf.Session()
            
            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #
            summary_writer = tf.summary.FileWriter(log_files_path+'multi_da_layer/', sess.graph)
        
            #initialization of all the variables
            init_op = tf.global_variables_initializer()
            sess.run(init_op)
        
            #will work with this later
            #saver.restore(sess, log_files_path+'multi_layer/model-checkpoint-66000')
            
            loss_trace = []

            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                total_batch = int(Xtrain.shape[0]/batch_size)
            
                # Loop over all batches
                for i in range(total_batch):

                    minibatch_x = Xtrain[i*batch_size: (i+1)*batch_size ]
                    minibatch_y = ytrain[i*batch_size: (i+1)*batch_size ]
                    # Fit training using batch data
                    #the training is done using the training dataset
                    sess.run(train_op, feed_dict={x: minibatch_x, y: minibatch_y})
                    # Compute average loss of all batches
                    avg_cost += sess.run(cost, feed_dict={x: minibatch_x, y: minibatch_y})/total_batch
                    
                # Display logs per epoch step
                if epoch % display_step == 0:
                    
                    #the accuracy is evaluated using the validation dataset
                    accuracy = sess.run(eval_op, feed_dict={x: Xval, y: yval})
                    loss_trace.append(1-accuracy)    
                    print("Epoch:", '%03d' % (epoch+1), "cost function=", "{:0.7f}".format(avg_cost), " Validation Error:", (1.0 - accuracy))
                    summary_str = sess.run(summary_op, feed_dict={x: minibatch_x, y: minibatch_y})
                    summary_writer.add_summary(summary_str, sess.run(global_step))
                        
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    saver.save(sess, log_files_path+'multi_da_layer/model-checkpoint', global_step=global_step)
                        
            print("Optimization Finished!")
            #accuracy evaluated with the whole test dataset
            accuracy = sess.run(eval_op, feed_dict={x: Xtest, y: ytest})
            print("Test Accuracy:", accuracy)
                    
            elapsed_time = time.time() - start_time
            print('Execution time (seconds) was %0.3f' % elapsed_time)
            
            # Visualization of the results
            # loss function
            #plt.plot(loss_trace)
            #plt.title('Cross Entropy Loss')
            #plt.xlabel('epoch')
            #plt.ylabel('loss')
            #plt.show()

Weight Matrix: <tf.Variable 'multi_da_layer/hidden_layer_1/W:0' shape=(513, 1000) dtype=float32_ref>
Bias Vector: <tf.Variable 'multi_da_layer/hidden_layer_1/b:0' shape=(1000,) dtype=float32_ref>
Weight Matrix: <tf.Variable 'multi_da_layer/hidden_layer_2/W:0' shape=(1000, 600) dtype=float32_ref>
Bias Vector: <tf.Variable 'multi_da_layer/hidden_layer_2/b:0' shape=(600,) dtype=float32_ref>
Weight Matrix: <tf.Variable 'multi_da_layer/hidden_layer_3/W:0' shape=(600, 300) dtype=float32_ref>
Bias Vector: <tf.Variable 'multi_da_layer/hidden_layer_3/b:0' shape=(300,) dtype=float32_ref>
Weight Matrix: <tf.Variable 'multi_da_layer/hidden_layer_4/W:0' shape=(300, 100) dtype=float32_ref>
Bias Vector: <tf.Variable 'multi_da_layer/hidden_layer_4/b:0' shape=(100,) dtype=float32_ref>
Weight Matrix: <tf.Variable 'multi_da_layer/hidden_layer_5/W:0' shape=(100, 25) dtype=float32_ref>
Bias Vector: <tf.Variable 'multi_da_layer/hidden_layer_5/b:0' shape=(25,) dtype=float32_ref>
Weight Matrix: <tf.Variable '